In [1]:
import numpy as np
import math
from sympy import *
from sympy.abc import x
from interval import imath
from interval import fpu
from interval import interval

Напишем алгоритм, который будет находить экстремум функции вдоль заданного направления.
Алгоритм будет основываться на методе "Золотого сечения" так как является наиболее эффективным по стравнению с основными методами одномерной оптимизации.

In [2]:
def argmin(f, i, D, p):
    d = D[i]
    a = d[0]
    b = d[1]
    phi = (1 + math.sqrt(5)) / 2
    x_1 = b - (b - a) / phi
    x_2 = a + (b - a) / phi
    p_1 = p.copy()
    p_2 = p.copy()
    p_1[i] = x_1
    p_2[i] = x_2
    f_1 = f(p_1)
    f_2 = f(p_2)
    while b - a > 0.001:
        if f_1 <= f_2:
            b = x_2
            x_2 = x_1
            x_1 = b - (b - a) / phi

            p_1[i] = x_1
            p_2[i] = x_2

            f_2 = f_1
            f_1 = f(p_1)
        else:
            a = x_1
            x_1 = x_2
            x_2 = a + (b - a) / phi

            p_1[i] = x_1
            p_2[i] = x_2

            f_1 = f_2
            f_2 = f(p_2)
    p_ = p_1
    for i in range(len(p_)):
        p_[i] = (p_1[i] + p_2[i]) / 2
    return p_

Функция вычисляющая евклидово расстояние между 2 точками

In [3]:
def r(p_1, p_2):
    r_ = 0
    for i in range(len(p_1)):
        r_ += (p_1[i] - p_2[i]) ** 2
    return math.sqrt(r_)

Функция принимающая функцию, множество определения и начальную точку.
Она раелизует метод наискорейшего спуска, последовательно ищет минимум вдоль направлений параллельных координатным осям и останавливается когда улучшение точности становится малым.

In [4]:
def fast_search(f, D, p):
    dimension = len(D)

    while True:
        p_0 = p
        for i in range(dimension):
            p = argmin(f, i, D, p)
        if r(p_0, p) < 0.01:
            break
    return p

Рассмотрим пример работы функции на примере:

In [5]:
def F(x):
    return (x[0] ** 2) / 2 - (x[1] ** 2) / 4 + 3

p = fast_search(F, [[-10, 10], [-10, 10]], [1, 1])
y = F(p)
print(p)
print(y)

[-2.163525435194824e-16, -9.999591436509917]
-21.997957224280615


Попробуем теперь применить тут алгоритм Moore-Skelboe для поиска минимума вдоль направления:

In [6]:
def fast_search1(f, D, p):
    dimension = len(D)

    while True:
        p_0 = p
        for i in range(dimension):
            p = Moore_Skelboe(f, i, D, p)
        if r(p_0, p) < 0.01:
            break
    return p

In [7]:
def Moore_Skelboe(f, index, D, p):
    e = 0.001
    interval_d = D.copy()
    for i in range(len(p)):
        interval_d[i] = interval[p[i], p[i]]
    interval_d[index] = D[index]

    interval_e = f(interval_d)
    list_of_intervals = [[interval_d, interval_e]]
    U = fpu.max(interval_e)[1]
    w = len(interval_e)
    best_interval = list_of_intervals[0]
    while w > e:
        list_of_intervals.pop(0)
        mid = (fpu.max(best_interval[0][index])[0] + fpu.max(best_interval[0][index])[1]) / 2
        interval_1 = best_interval[0].copy()
        interval_2 = best_interval[0].copy()
        interval_1[index] = interval[fpu.max(best_interval[0][index])[0], mid]
        interval_1e = f(interval_1)
        interval_2[index] = interval[mid, fpu.max(best_interval[0][index])[1]]
        interval_2e = f(interval_2)
        U = min(U, fpu.max(interval_1e)[1])
        U = min(U, fpu.max(interval_2e)[1])
        list_of_intervals.append([interval_1, interval_1e])
        list_of_intervals.append([interval_2, interval_2e])
        for el in list_of_intervals:
            if U < fpu.max(el[1])[0]:
                list_of_intervals.remove(el)
        list_of_intervals.sort(key=lambda item: fpu.max(item[1])[0])
        best_interval = list_of_intervals[0]
        w = fpu.max(best_interval[0][index])[1] - fpu.max(best_interval[0][index])[0]
    return fpu.max(list_of_intervals[0][1])[0]

In [8]:
p = fast_search1(F, [[-10, 10], [-10, 10]], [1, 1])
y = F(p)
print(p)
print(y)

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'